

# Challenge Ingredion - Sprint 2
Este notebook foi desenvolvido para o Challenge em parceria com a Ingredion, na Sprint 2.

Conectando o caminho do dataset diretamente do Drive.

*(Há a opção de pegarmos as bases de dados e subir nas pastas do notebook, assim rodando localmente.)*

In [17]:
dataset = "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/dado-estruturados.csv"

Definindo o código para conexão com o Google Drive.

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Importantos as libs necessárias.

In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Análise exploratória dos dados

Aqui damos início a nossa análise exploratória dos dados para identificarmos nossas variáveis chaves do projeto.

In [ ]:
df_structured = pd.read_csv(dataset)

df_structured.shape

Definindo a variável do dataframe e **exibindo as 5 primeiras linhas** e **as últimas 5 linhas** do df.



In [ ]:
df_structured.head()

In [ ]:
df_structured.tail()

Obtém **informações gerais** de cada coluna.

In [ ]:
df_structured.describe()

Verifica colunas que possui **valores nulos**.

In [ ]:
df_structured.notnull()

Conta **se há colunas nulas**.

In [ ]:
df_structured.isnull().sum()

Conta as colunas **duplicadas**.

In [ ]:
df_structured.duplicated().sum()


### Identificando Outliers

Após uma breve análise exploratória, vamos tentar identificar os outliers do nosso dataset.

Utilizando o gráfico **Boxplot** para identificar outliers.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_structured.select_dtypes(include=['number']))
plt.xticks(rotation=45)
plt.title("Boxplot das variáveis para identificação de outliers")
plt.show()

Podemos ver que a **Quantidade Produzida (Toneladas)** aprensenta um número maior de outliers. Outras variáveis também apresentam, mas são outliers quase irrelevantes.

Usando a **Matriz de Correlação Visual** para entender os padrões, utilizando Seaborn.




In [ ]:
df_apenas_numericos = df_structured.select_dtypes(include=['number'])

plt.figure(figsize=(10, 6))
sns.heatmap(df_apenas_numericos.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.show()

**Forte correlação entre:**

* Ano e valor da produção **(0.89)**.
* Área plantada ou destinada à colheita (hectares) e Percentual Geral **(0.89)**.
* Área colhida e área plantada ou destinada à colheita **(0.75)**.
* Quantidade produzida (Toneladas) e Rendimento médio da produção **(0.92)**.

**Correlação Fraca com o Rendimento:**

* Área plantada ou destinada à colheita (hectares) e Valor da produção (-0.41).


Utilizando a regra do **IQR (Intervalo Interquatil)** para remover outliers.

In [ ]:
variaveis_importantes = ["Rendimento médio da produção (Quilogramas por Hectare)",
                         "Quantidade produzida (Toneladas)",
                         "Área colhida (Hectares)",
                         "Valor da produção (Mil Reais)",
                        ]

df_filtrado = df_structured.copy()

for coluna in variaveis_importantes:
    Q1 = df_filtrado[coluna].quantile(0.25)
    Q3 = df_filtrado[coluna].quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    df_filtrado = df_filtrado[df_filtrado[coluna].between(limite_inferior, limite_superior)]

df_filtrado.shape
df_filtrado

Agora posssuímos um Dataframe **filtrado** e informações mais concretas.

### Aplicação da Técnica de Segmentação - SAM



In [ ]:
!pip install torch torchvision albumentations opencv-python matplotlib numpy rasterio

In [ ]:
# Criar symlink para o dataset
!ln -s '/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/not-structured'

In [ ]:
# Celda 3: Classe Dataset
import cv2
import os
import albumentations as A
import torch
from torch.utils.data import Dataset, DataLoader

class SatelliteDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

        # Verifique se todos os arquivos têm máscaras correspondentes
        for img_name in self.images:
            mask_name = img_name.replace(".jpg", "_mask.png")
            mask_path = os.path.join(mask_dir, mask_name)
            if not os.path.exists(mask_path):
                raise FileNotFoundError(f"Máscara não encontrada: {mask_path}")

    def __len__(self): # Added __len__ method
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_name = img_name.replace(".jpg", "_mask.png")
        mask_path = os.path.join(self.mask_dir, mask_name)

        # Leitura com verificação explícita
        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Imagem não pode ser lida: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise ValueError(f"Máscara não pode ser lida: {mask_path}")

        mask = mask / 255.0  # Divisão por float

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

# Transformações
train_transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

val_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# Celda 4: Criação dos DataLoaders
batch_size = 4
num_epochs = 10

train_dataset = SatelliteDataset(
    "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/train/images",
    "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/train/masks",
    train_transform
)

val_dataset = SatelliteDataset(
    "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/val/images",
    "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/val/masks",
    val_transform
)

test_dataset = SatelliteDataset(
    "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/test/images",
    "/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/test/masks",
    val_transform
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=1)

In [ ]:
# Celda 5: Modelo DeepLabV3+
from torchvision.models.segmentation import deeplabv3_resnet50

model = deeplabv3_resnet50(pretrained=True)
model.classifier[4] = torch.nn.Conv2d(256, 1, kernel_size=1)  # 1 classe

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Celda 6: Função de cálculo de IoU (ajustada para NumPy)
def calculate_iou(pred, target):
    # Verifica se os inputs são tensores do PyTorch
    is_tensor = isinstance(pred, torch.Tensor)

    # Converte para booleanos
    if is_tensor:
        pred = pred.bool()       # Método do PyTorch
        target = target.bool()   # Método do PyTorch
        intersection = torch.logical_and(pred, target).sum()
        union = torch.logical_or(pred, target).sum()
    else:
        pred = pred.astype(bool)    # Método do NumPy
        target = target.astype(bool)  # Método do NumPy
        intersection = np.logical_and(pred, target).sum()
        union = np.logical_or(pred, target).sum()

    # Calcula IoU e converte para float (evita erro de tipo)
    iou = (intersection / union) if union != 0 else 0.0
    return iou.item() if is_tensor else iou  # .item() para tensores

# Celda 7: Loop de Treinamento
best_iou = 0.0

for epoch in range(num_epochs):
    # Treino
    model.train()
    train_loss = 0.0
    for images, masks in train_loader:
        images = images.permute(0, 3, 1, 2).float().to(device)
        masks = masks.float().to(device)

        if images.shape[0] == 1:  # Check if batch size is 1
            # Pad the batch to increase batch size
            images = torch.cat([images] * 2, dim=0) # Duplicate the image to create a batch of 2
            masks = torch.cat([masks] * 2, dim=0) # Duplicate the mask to create a batch of 2

        outputs = model(images)['out']
        loss = criterion(outputs.squeeze(1), masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validação
    model.eval()
    val_loss = 0.0
    val_iou = 0.0
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.permute(0, 3, 1, 2).float().to(device)
            masks = masks.float().to(device)

            outputs = model(images)['out']
            val_loss += criterion(outputs.squeeze(1), masks).item()

            preds = (torch.sigmoid(outputs) > 0.5).float()
            val_iou += calculate_iou(preds.cpu(), masks.cpu())

    # Salvar melhor modelo
    avg_val_iou = val_iou / len(val_loader)
    if avg_val_iou > best_iou:
        best_iou = avg_val_iou
        torch.save(model.state_dict(), '/content/best_model.pth')

    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f} | Val IoU: {avg_val_iou:.4f}')

In [ ]:
# Celda 8: Carregar melhor modelo
model.load_state_dict(torch.load('/content/best_model.pth'))
model.eval()

# Celda 9: Teste
test_iou = 0.0
test_loss = 0.0
with torch.no_grad():
    for images, masks in test_loader:
        images = images.permute(0, 3, 1, 2).float().to(device)
        masks = masks.float().to(device)

        outputs = model(images)['out']
        test_loss += criterion(outputs.squeeze(1), masks).item()

        preds = (torch.sigmoid(outputs) > 0.5).float()
        test_iou += calculate_iou(preds.cpu(), masks.cpu())

print(f'\nTest Loss: {test_loss/len(test_loader):.4f} | Test IoU: {test_iou/len(test_loader):.4f}')

In [ ]:
# Celda 10: Visualização
import matplotlib.pyplot as plt
import numpy as np
import torch

idx = 0
image, mask = test_dataset[idx]  # A imagem original está em (H, W, C)

# Aplicar transformações na imagem no formato (H, W, C)
transformed = val_transform(image=image, mask=mask)
transformed_image = transformed['image']  # Agora (H, W, C) após transformações

# Converter para tensor e ajustar dimensões
input_tensor = torch.from_numpy(transformed_image).permute(2, 0, 1)  # (C, H, W)
input_tensor = input_tensor.unsqueeze(0).float().to(device)  # Adicionar dimensão de batch

with torch.no_grad():
    output = model(input_tensor)['out'].squeeze()
pred_mask = (torch.sigmoid(output) > 0.5).cpu().numpy()

# Plot
plt.figure(figsize=(15,5))

# Imagem Original (desnormalize se necessário)
plt.subplot(1,3,1)
original_image = image.numpy() if isinstance(image, torch.Tensor) else image  # Garante que é um array NumPy
plt.imshow(original_image)
plt.title('Imagem Original')

# Máscara Real
plt.subplot(1,3,2)
plt.imshow(mask, cmap='gray')
plt.title('Máscara Real')

# Predição
plt.subplot(1,3,3)
plt.imshow(pred_mask, cmap='gray')
plt.title(f'Predição (IoU: {calculate_iou(pred_mask, mask):.2f})')
plt.show()

In [ ]:
# Celda 11: Função para predição
def predict_image(image_path):
    # Verifica se a imagem existe
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Arquivo não encontrado: {image_path}")

    # Carrega a imagem
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Falha ao ler a imagem: {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converte para RGB

    # Aplica as transformações e converte para tensor
    augmented = val_transform(image=image)
    input_tensor = augmented['image']  # Assume-se que é um array NumPy

    # Converte para tensor do PyTorch e ajusta as dimensões
    input_tensor = torch.from_numpy(input_tensor).float()  # Converte para tensor
    input_tensor = input_tensor.permute(2, 0, 1)          # Muda de (H, W, C) para (C, H, W)
    input_tensor = input_tensor.unsqueeze(0).to(device)    # Adiciona dimensão do batch

    # Predição
    with torch.no_grad():
        output = model(input_tensor)['out'].squeeze()

    return (torch.sigmoid(output) > 0.5).cpu().numpy()

# Exemplo de uso
prediction = predict_image('/content/drive/MyDrive/Colab Notebooks/FIAP/Fase 6/Atividade - Challenge/data/not-structured/img-satelite-fazenda-yrere.png')
prediction

## Desenvolvendo os Modelos Preditivos
Agora vamos desenvolver 5 tipos de modelos preditivos para prever o rendimento da safra.

Foi usado os seguintes modelos:
- Ridge;
- Regressão Linear;
- Árvore de Regressão;
- Floresta de Regressão;
- KNN de Regressão;
- SVR;
- GradientBoostingRegressor;

Padronizando os dados com o **RobustScaler.**

In [ ]:
X = df_filtrado.drop(['Yield', 'Crop'], axis=1)
y = df_filtrado['Yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
ridge_model = Ridge(alpha=0.01, fit_intercept=True, solver='auto')
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

print("Ridge Regressão")
print("Erro Quadrático Médio:", mean_squared_error(y_test, y_pred_ridge))
print("R-quadrado:", r2_score(y_test, y_pred_ridge))

In [ ]:
linear_model = LinearRegression(fit_intercept=True, copy_X=False, n_jobs=-1)
linear_model.fit(X_train, y_train)

y_pred_linear = linear_model.predict(X_test)
print("Regressão Linear")
print("Erro Quadrático Médio:", mean_squared_error(y_test, y_pred_linear))
print("R-quadrado:", r2_score(y_test, y_pred_linear))

In [ ]:
arvore_decisao = DecisionTreeRegressor(max_depth=1, min_samples_split=2, min_samples_leaf=10, random_state=42)
arvore_decisao.fit(X_train, y_train)
y_pred_arvore = arvore_decisao.predict(X_test)

print("Árvore de Decisão")
print("Erro Quadrático Médio:", mean_squared_error(y_test, y_pred_arvore))
print("R-quadrado:", r2_score(y_test, y_pred_arvore))

In [ ]:
floresta_aleatoria = RandomForestRegressor(n_estimators=100, max_depth=4, min_samples_split=2, min_samples_leaf=1, random_state=42)
floresta_aleatoria.fit(X_train, y_train)

y_pred_floresta = floresta_aleatoria.predict(X_test)
print("Floresta Aleatória")
print("Erro Quadrático Médio:", mean_squared_error(y_test, y_pred_floresta))
print("R-quadrado:", r2_score(y_test, y_pred_floresta))

In [ ]:
knn_model = KNeighborsRegressor(n_neighbors=1, weights='uniform', algorithm='brute', p=1)
knn_model.fit(X_train, y_train)

y_pred_knn = knn_model.predict(X_test)
print("K-Nearest Neighbors")
print("Erro Quadrático Médio:", mean_squared_error(y_test, y_pred_knn))
print("R-quadrado:", r2_score(y_test, y_pred_knn))

In [ ]:
svr_model = SVR(kernel='sigmoid', C=1.0, epsilon=0.4, gamma='scale')
svr_model.fit(X_train, y_train)

y_pred_svr = svr_model.predict(X_test)
print("Support Vector Regression")
print("Erro Quadrático Médio:", mean_squared_error(y_test, y_pred_svr))
print("R-quadrado:", r2_score(y_test, y_pred_svr))

In [ ]:
gb_model = GradientBoostingRegressor(min_samples_split=10, learning_rate=0.03, random_state=42)
gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)
print("Gradient Boosting")
print("Erro Quadrático Médio:", mean_squared_error(y_test, y_pred_gb))
print("R-quadrado:", r2_score(y_test, y_pred_gb))

## Conclusão

Os gráficos mostra **a relação entre os valores reais** e as **previsões de rendimento do modelo de Regressão Linear** e o **modelo Ridge Regression**. Embora a maioria das previsões esteja próxima dos valores reais, há um ponto fora da curva, indicando um possível outlier ou uma limitação do modelo.

O uso do RobustScaler **ajudou a minimizar o impacto dos outliers**. No geral, os modelos de Regressão Linear e Ridge Regression apresentaram desempenho estável, mantendo um coeficiente de determinação (R²) **acima de 80%**.

In [ ]:
plt.scatter(y_test, y_pred_linear)
plt.xlabel("Valores Reais (Rendimento da Safra)")
plt.ylabel("Valores Previstos")
plt.title("Ridge Regression - Rendimento Real vs. Previsto")
plt.show()

In [ ]:
plt.scatter(y_test, y_pred_ridge)
plt.xlabel("Valores Reais (Rendimento da Safra)")
plt.ylabel("Valores Previstos")
plt.title("Ridge Regression - Rendimento Real vs. Previsto")
plt.show()